# Google Play Store scrapper

Use this notebook to scrap google playstore and save the results as MongoDB Collection.
Download Chrome dirver and paste it in the current working directory. - this Script tested with Mac & windows chrome driver

In [1]:
#install all required libraries 
!pip install selenium
!pip install pymongo

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
#lets import all required packages
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
from selenium.webdriver.common.by import By
import csv

In [3]:
#We will use Chorme driver - Copy chromedriver to the current directory. if different directory then use that path

#if its windows uncomment below section and create driver
#drive=webdriver.Chrome("chromedriver")

#below section is for MacOS
PROJECT_ROOT = os.path.abspath(os.getcwd())
DRIVER_BIN = os.path.join(PROJECT_ROOT, "chromedriver")

browser = webdriver.Chrome(executable_path = DRIVER_BIN)

In [4]:
#Set timeout - Be patient and gentle to the website
browser.set_page_load_timeout(10)

In [5]:
#here we will give the list of URLs we need to iterate over
URLList = ['https://play.google.com/store/apps/details?id=com.orbital.brainiton',
          'https://play.google.com/store/apps/details?id=com.jetstartgames.chess',
          'https://play.google.com/store/apps/details?id=com.easybrain.sudoku.android']

Below we have some usefull Methods that we will use it for this program

In [6]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath,browser):
    try:
        browser.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [7]:
def create_css_selector(compound_class):
    return '.' + '.'.join(compound_class.split(' '))

In [8]:
def element_exists_by_css_selector(class_str, browser):
    try:
        print(class_str + " TO " + create_css_selector(class_str))
        return browser.find_elements(By.CSS_SELECTOR, create_css_selector(class_str))
    except NoSuchElementException:
        print('//////////////////AAWWW ERROR :<////////////////')
        return None

In [9]:
def scroll_to_div(class_str, browser):
    element = browser.find_element(By.CLASS_NAME, class_str)

    actions = ActionChains(browser)
    actions.move_to_element(element).perform()

In [10]:
def click_on_show_more(browser):
    banana_q = element_exists_by_css_selector('U26fgb O0WRkf oG5Srb C0oVfc n9lfJ', browser)
    if  banana_q is not None:
        for banana in banana_q:
            if banana.is_displayed():
                print('=============SHOW MORE=============')
                banana.click()

Below are some of the contants we have identified for each of the tags. If the program breaks check and update with new constants

In [11]:
review_section_classname = "d15Mdf bAhLNe"
helpful_count_classname = "jUL89d y92BAb"
rating_count_classname = "vQHuPe bUWb7c"
date_classname = "p2TkOb"
full_review_classname = "fbQN7e"    
teaser_review_classname = "bN97Pc"

In [12]:
#connection details to local MongoDB
mongo_client = MongoClient()
db = mongo_client.walkietechie
collection = db.google_play_reviews

Main Method to iterate and list of URLs and start the scrapping

In [17]:
w = csv.writer(open('reviews.csv','w'))
reviewWriter = csv.writer(open('reviews.csv', 'wb'), delimiter=' ',quotechar=' ', quoting=csv.QUOTE_MINIMAL)
for url in URLList:
    browser.get(url)
    print("Now reading ->", url)
    time.sleep(10)
    #Lets try to click the "Read All Reviews"
    #//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[6]/div/content/span
    xpath=browser.find_element_by_xpath(("//*[@id='fcxH9b']/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[6]/div/content/span"));
    print(xpath)
    xpath.click()
    time.sleep(10)
    #now lets try to scroll down the page - This page load when we scroll down for more results
    SCROLL_PAUSE_TIME = 10
    i = 0 # later we can change it to some other bigger number
    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        #lets try to click the full review button if its visible
        full_review_path = "//*[@id='fcxH9b']/div[4]/c-wiz[2]/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div/div[9]/div/div[2]/div[2]/span[1]/div/button"
        theButtons = browser.find_elements_by_xpath("//*[contains(@class,'LkLjZd ScJHi OzU4dc')]");
    
        click_on_show_more(browser)

        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        scroll_to_div('W4P4ne', browser)
        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        i = i + 1
        if new_height == last_height or i == 500: # or i == 10
            print('Will exit here')
            break
        last_height = new_height
        
    #lets try to get the Name of the APP First
    data = browser.page_source
    soup = BeautifulSoup(data, "html.parser")
    #Get H1
    appTitleSpan = soup.find("h1",{"itemprop":"name"})
    appTitleName = appTitleSpan.find("span").text
    print(appTitleName)
    reivewSpan = soup.find_all("div",{"class":review_section_classname})
    
    for item in reivewSpan:
        review = {}

        helpful_count = item.find("div", {"class":helpful_count_classname})
        rating_count = item.find_all("div", {"class":rating_count_classname})
        date = item.find("span", {"class":date_classname})    
        full_review = item.find("span", {"jsname":full_review_classname})
        teaser_review = item.find("span", {"jsname":teaser_review_classname})
        review['app_name'] = appTitleName
        review["helpful_count"] = helpful_count.get_text()
        review["rating_count"] = len(list(rating_count))
        review["date"] = date.get_text()
    
        if full_review is not None:
            review["full_review"] = full_review.get_text()
        review["teaser_review"] = teaser_review.get_text()
    
        print(review)
        
        collection.insert_one(review)
        #reviewWriter.writerow("%s, %s, %s, %s, %s, %s\n" % \
        #       (str(review['app_name']), str(review['helpful_count']), str(review['rating_count']),str(review['date']),str(review['full_review']),str(review['teaser_review'])))
    

Now reading -> https://play.google.com/store/apps/details?id=com.orbital.brainiton
<selenium.webdriver.remote.webelement.WebElement (session="488396206f3c3dd587ddb25a9a06a84d", element="0.355071981771502-1")>
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
=============SHOW MORE=============
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
=============SHOW MORE=============
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ

U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
=============SHOW MORE=============
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
=============SHOW MORE=============
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
=============SHOW MORE=============
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
U26fgb O0WRkf oG5Srb C0oVfc n9lfJ TO .U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ
=============SHOW MORE======

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=63270): Max retries exceeded with url: /session/488396206f3c3dd587ddb25a9a06a84d/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001E7A88E46D8>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it',))

In [14]:
browser.quit()